<a href="https://colab.research.google.com/github/kamrulhasan04947/mlprac/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
import nltk
from sklearn.pipeline import make_pipeline

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
from tables.description import Description
data = pd.read_csv('/content/drive/MyDrive/Data sets/archive (5)/train.txt', delimiter=';', names=['Description', 'Emotion'])

In [ ]:
data.head()

,Description,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
data.columns

Index(['Description', 'Emotion'], dtype='object')

In [ ]:
import re

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelenc = LabelEncoder()

In [ ]:
stemmer = WordNetLemmatizer()
def text_processor(text):
  stop_words = set(stopwords.words('english'))
  word_tokens = nltk.word_tokenize(re.sub(r'[^\w\s]','',text))
  filtered = [stemmer.lemmatize(word) for word in word_tokens if word not in stop_words]
  return ' '.join(filtered)

In [ ]:
def procesedDf(df):
    df['clened_Description'] = df['Description'].apply(text_processor)
    df['Encoded_Emotion'] =labelenc.fit_transform(df['Emotion'])
    df_cleaned = df[['clened_Description', 'Encoded_Emotion']].copy()
    return pd.DataFrame(data=df_cleaned , columns=df_cleaned .columns)


In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
procesed_data = procesedDf(data)


In [ ]:
procesed_data.head(200)

,clened_Description,Encoded_Emotion
0,didnt feel humiliated,4
1,go feeling hopeless damned hopeful around some...,4
2,im grabbing minute post feel greedy wrong,0
3,ever feeling nostalgic fireplace know still pr...,3
4,feeling grouchy,0
...,...,...
195,died would alex matt feel regretful coming visit,4
196,feel educating family supporting educationg ma...,2
197,im sure lost feel like everyone hostile toward...,0
198,feel like boring sewing make since easy nothin...,4


In [ ]:
x= procesed_data['clened_Description']
y= procesed_data['Encoded_Emotion']

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

In [ ]:
tdvec = TfidfVectorizer()

In [ ]:
rf = RandomForestClassifier(max_depth=100, random_state=10)
rf_cl_pipline = make_pipeline(tdvec, rf)
fit_data = rf_cl_pipline.fit(x_train, y_train)
prediction = fit_data.predict(x_test)
classificationReport = classification_report(y_test, prediction)
print(classificationReport)


              precision    recall  f1-score   support

           0       0.88      0.71      0.78       665
           1       0.85      0.73      0.78       635
           2       0.73      0.96      0.83      1751
           3       0.91      0.58      0.71       477
           4       0.92      0.86      0.89      1559
           5       0.88      0.62      0.73       193

    accuracy                           0.83      5280
   macro avg       0.86      0.74      0.79      5280
weighted avg       0.84      0.83      0.82      5280



In [ ]:
grid_param = {'C': [0.1, 0.01, 0.1, 1, 100],
              'gamma': [1, 0.75, 0.5, 0.25],
              'kernel': ['rbf', 'poly', 'linear']}
grid = GridSearchCV(SVC(), grid_param,  refit=True,  verbose=3)
rf_cl_pipline = make_pipeline(tdvec, grid)
rf_cl_pipline.fit(x, y)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.363 total time=  47.3s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.370 total time=  43.4s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.374 total time=  44.2s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.368 total time=  42.9s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.374 total time=  47.6s
[CV 1/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.335 total time=  40.8s
[CV 2/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.335 total time=  40.3s
[CV 3/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.335 total time=  40.4s
[CV 4/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.336 total time=  41.1s
[CV 5/5] END .......C=0.1, gamma=1, kernel=poly;, score=0.335 total time=  40.7s
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.628 total time=  32.7s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('gridsearchcv',
                 GridSearchCV(estimator=SVC(),
                              param_grid={'C': [0.1, 0.01, 0.1, 1, 100],
                                          'gamma': [1, 0.75, 0.5, 0.25],
                                          'kernel': ['rbf', 'poly', 'linear']},
                              verbose=3))])